<a href="https://colab.research.google.com/github/vaibhavrouduri/Thesis-Work/blob/main/Vaibhav_Code/Nearest%20Neighbors/Nearest_Neighbors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import random
import time
from itertools import filterfalse
from scipy import spatial
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from gensim.scripts.glove2word2vec import glove2word2vec

# Glove Common Crawl

In [ ]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip # Retrieves content from the given URL
!unzip -q glove.840B.300d.zip # Unzips the file and returns the .txt file which can be used for analysis

In [ ]:
embeddings_glove = {} # Empty dictionary in which the words in the vocabulary and the corresponding vector will be added
with open("glove.840B.300d.txt") as f:
    for line in f: # Each line in the file contains the token followed by the vector representation of that token trained in glove in 300 dimensions separated by a space. Each dimension of the vector is separated by a space as well
        word, coefs = line.split(maxsplit=1) # Splits the line as described above at the first space, hence the token gets separated from the vector. The token is stored in 'word' and the vector is stored in 'coefs'
        coefs = np.fromstring(coefs, "f", sep=" ") # Returns the vector representation in the form of an array which can now be used for analysis (cosine similarity, etc)
        embeddings_glove[word] = coefs # Add the token and its corresponding vector into the dictionary as key and value pair. Now we can call the vector of any token in the vocabulary via the dictionary

# Glove Wikipedia

In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip # Retrieves content from the given URL
!unzip -q glove.6B.zip # Unzips the file and returns the .txt file which can be used for analysis. In this case, returns 4 files, having 50, 100, 200, and 300 dimensional vectors. We are choosing to use the 300 dimensional vectors

--2021-11-29 19:46:57--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-29 19:46:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-29 19:46:57--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [5]:
embeddings_glove_wiki_gw = {} # Empty dictionary in which the words in the vocabulary and the corresponding vector will be added
with open("glove.6B.300d.txt") as f:
    for line in f: # Each line in the file contains the token followed by a the vector representation of that token trained in glove in 300 dimensions separated by a space. Each dimension of the vector is separated by a space as well
        word, coefs = line.split(maxsplit=1) # Splits the line as described above at the first space, hence the token gets separated from the vector. The token is stored in 'word' and the vector is stored in 'coefs'
        coefs = np.fromstring(coefs, "f", sep=" ") # Returns the vector representation in the form of an array which can now be used for analysis (cosine similarity, etc)
        embeddings_glove_wiki_gw[word] = coefs # Add the token and its corresponding vector into the dictionary as key and value pair. Now we can call the vector of any token in the vocabulary via the dictionary

# Fastext Common Crawl

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip # Retrieves content from the given URL
!unzip -q crawl-300d-2M.vec.zip # Unzips the file and returns the .vec file which can be used for analysis

In [ ]:
embeddings_fastext_cc = {}
with open("crawl-300d-2M.vec") as f:
    for line in f: # Each line in the file contains the token followed by a the vector representation of that token trained in glove in 300 dimensions separated by a space. Each dimension of the vector is separated by a space as well
        word, coefs = line.split(maxsplit=1) # Splits the line as described above at the first space, hence the token gets separated from the vector. The token is stored in 'word' and the vector is stored in 'coefs'
        coefs = np.fromstring(coefs, "f", sep=" ") # Returns the vector representation in the form of an array which can now be used for analysis (cosine similarity, etc)
        embeddings_fastext_cc[word] = coefs # Add the token and its corresponding vector into the dictionary as key and value pair. Now we can call the vector of any token in the vocabulary via the dictionary

# Fastext Wikipedia

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip # Retrieves content from the given URL
!unzip -q wiki-news-300d-1M.vec.zip # Unzips the file and returns the .vec file which can be used for analysis

In [ ]:
embeddings_fastext_wiki = {}
with open("wiki-news-300d-1M.vec") as f:
    for line in f: # Each line in the file contains the token followed by a the vector representation of that token trained in glove in 300 dimensions separated by a space. Each dimension of the vector is separated by a space as well
        word, coefs = line.split(maxsplit=1) # Splits the line as described above at the first space, hence the token gets separated from the vector. The token is stored in 'word' and the vector is stored in 'coefs'
        coefs = np.fromstring(coefs, "f", sep=" ") # Returns the vector representation in the form of an array which can now be used for analysis (cosine similarity, etc)
        embeddings_fastext_cc[word] = coefs # Add the token and its corresponding vector into the dictionary as key and value pair. Now we can call the vector of any token in the vocabulary via the dictionary

# word2vec

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
embeddings_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# Nearest Neighbors

In [9]:
def cos(v1, v2): 
  """Returns the cosine similarity of 2 vectors of same dimesionality.

    Arguments:
    v1 -- Vector 1
    v2 -- Vector 2
    """

  num = np.dot(v1, v2)
  denom = np.linalg.norm(v1) * np.linalg.norm(v2)
  return num/denom

In [10]:
def vocab_to_list(embedding_type):
    """Returns the list of words in the vocabulary of a chosen embedding type(eg. glove with wikipedia)

    Arguments:
    embedding_type -- The dictionary for the corpus algorithm pair being used in the current analysis (eg. for glove with common crawl, use embeddings_glove)
    """
  words = []
  for w in embedding_type:
    words.append(w)
  return words

In [11]:
def similar_words(word, number, embedding_type):
  """Returns a chosen number of most similar words to a given word in terms of cosine similarity.

    Arguments:
    word -- The token whose most similar words you want to find
    number -- The number of similar words you want to find
    """
  similar_word = []

  words_for_word = vocab_to_list(embedding_type).copy()

  for w in words_for_word:
    if embedding_type[word].shape != embedding_type[w].shape:
      words_for_word.remove(w)

  for w in words_for_word: # For some reason unless I run this code snippet twice all the words with different vector shapes aren't getting removed.
    if embedding_type[word].shape != embedding_type[w].shape:
      words_for_word.remove(w)

  for w in words_for_word:
    similar_word.append(cos(embedding_type[word], embedding_type[w]))
  
  similar_word_copy = similar_word.copy()

  similar_word.sort(reverse = True)

  relevant_indices = []

  for i in range(number + 1):
    relevant_indices.append(similar_word_copy.index(similar_word[i]))

  ans = []

  for i in relevant_indices:
    ans.append(words_for_word[i])
  
  ans.remove(word)

  return ans

# Example

In [12]:
similar_words("skeptic", 10, embeddings_glove_wiki_gw)

['skeptics',
 'sceptic',
 'sceptics',
 'believer',
 'environmentalist',
 'proponent',
 'freethinker',
 'atheist',
 'skeptical',
 'ardent']